In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import xmltodict
import os
import json
import pickle
from lxml import etree


In [3]:
# mode = int(metadata['mode']) if metadata['mode'] is not None else 1
# beats_in_measure = int(metadata['beats_in_measure'])

# melody, chord = segments_parser(segments, mode, beats_in_measure)

In [4]:
from pathlib import Path

In [5]:
version = 'v7'
data_path = Path('data/midi')
version_path = data_path/version
orig_path = version_path/'midi_sources'

In [6]:
from fastai.data_block import get_files

In [7]:
h_path = orig_path/'hooktheory'

In [8]:
files = get_files(h_path, extensions=['.xml'], recurse=True); files[:10]

[PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wayne-sharpe/yu-gi-oh-theme-song/chorus.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wayne-sharpe/yu-gi-oh-theme-song/intro.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/what-a-day/kiefer/chorus.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/whiteflame/senbonzakura/pre-chorus.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/whiteflame/senbonzakura/verse.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/whiteflame/senbonzakura/chorus.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wham/last-christmas/verse.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wham/last-christmas/chorus.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wham/last-christmas/intro.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wham/freedom/chorus.xml')]

In [9]:
# Loading from specific file
# keywords = ['get-lucky', 'daft-punk', 'pre-chorus']
# keywords = ['skrillex', 'scary']
keywords = ['idina', 'verse', 'let']
# keywords = ['game-of-thrones', 'intro', 'ramin']
# keywords = ['kiss-from-a-rose', 'seal']
def contains_keywords(f): return all([k in str(f) for k in keywords])
search = [f for f in files if contains_keywords(f)]; search

[PosixPath('data/midi/v7/midi_sources/hooktheory/xml/i/idina-menzel/let-it-go/verse.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/i/idina-menzel/let-it-go/intro-and-verse.xml')]

In [10]:
from src.tab_parser import *

In [11]:
from src import roman_to_symbol
from src import to_pianoroll
from collections import defaultdict
from midi_data import keyc_offset

In [12]:
import music21

### Create config file

In [13]:

# sustain=True, sep_octave=True, sus_idx=2, hit_idx=1
config_opts = dict(note_octave=5, chord_octave=3,
                   continuous=True,
                   throw_err=False,
                   sus_idx=1, hit_idx=0,
                   bim=4, key=0, bpm=120, freq=4, 
                   bos_idx=-1, pad_idx=-3, none_idx=-2, enc_offset=3)

# none_idx is a rest - checked by cross_entropy, pad_idx is not countd in loss
class Config(object):
    def __init__(self, d): self.__dict__ = d
        
config = Config(config_opts)


### Constants

In [14]:

MODE_TO_KEYOFFSET = {
    '1': 0,
    '2': 2,
    '3': 4,
    '4': 5,
    '5': 7,
    '6': 9,
    '7': 11
}

PITCH_TO_SD = {
    0: '1',
    1: '1#',
    2: '2',
    3: '2#',
    4: '3',
    5: '4',
    6: '4#',
    7: '5',
    8: '5#',
    9: '6',
    10:'6#',
    11:'7',
}

SD_TO_PITCH = {v:k for k,v in PITCH_TO_SD.items()}



### Classes

In [15]:

from dataclasses import dataclass
import dataclasses
from typing import Dict, Any, AnyStr, List, Sequence, TypeVar, Tuple, Optional, Union


In [60]:
def parse(cls, d):
    cls_keys = cls.__dataclass_fields__.keys()
    kwargs = {key:d[key] for key in cls_keys}
    return cls(**kwargs)

@dataclass
class Base:
    @classmethod
    def from_dict(cls, d):
        cls_keys = cls.__dataclass_fields__.keys()
        kwargs = {key:d[key] for key in cls_keys}
        return cls(**kwargs)
    
    @classmethod
    def parse(cls, d):
        return cls.from_dict(d)

@dataclass
class HMetadata(Base):
    title:str
    BPM:str='120'
    beats_in_measure:str='4'
    key:str='C'
    mode:str='1'

@dataclass
class HBeat(Base):
    def __init__(self, abs:float, duration:float, measure:float, rel:float, **kwargs):
        self.abs = float(abs)
        self.duration = float(duration)
        self.measure = float(measure) if measure else None
        self.rel = float(rel) if rel else None
    @classmethod
    def parse_note(cls, d):
        return cls(abs=d['start_beat_abs'], 
                   duration=d['note_length'], 
                   measure=d['start_measure'],
                   rel=d['start_beat'])
    @classmethod
    def parse_chord(cls, d):
        return cls(abs=d['start_beat_abs'], 
                   duration=d['chord_duration'], 
                   measure=d['start_measure'],
                   rel=d['start_beat'])
    
    def end_time(self):
        return self.duration + self.abs
    
    def __repr__(self):
        return f'abs={self.abs}:dur={self.duration}'
        
@dataclass
class HPitch(Base):
    def __init__(self, pitch:int, octave:int, **kwargs):
        if config.throw_err: assert(pitch >= 0)
        if config.throw_err and octave: assert(octave >= 0)
        self.pitch = pitch
        self.octave = octave
    
    def sd(self):
        return PITCH_TO_SD[self.pitch]
    
    def abs(self, base_octave:int=0):
        if self.octave is None: return self.pitch
        return self.pitch + 12 * (self.octave+base_octave)
    
    def __repr__(self):
        return f'sd={self.sd()}:oct={self.octave}'
    
    @classmethod
    def parse_abs_pitch(cls, abs_pitch, base_octave=0):
        abs_pitch = int(abs_pitch)
        pitch = abs_pitch%12
        octave = abs_pitch//12+base_octave
        return cls(pitch=pitch, octave=octave)

@dataclass
class HNote(Base):
    def __init__(self, beat:HBeat, pitch:HPitch, **kwargs):
        self.beat = beat
        self.pitch = pitch
        
    def to_m21(self)->music21.note.Note:
        p = self.pitch.abs()
#         p = self.pitch.abs(base_octave=config.note_octave)
        n = music21.note.Note(p, quarterLength=self.beat.duration)
        return n, self.beat.abs
    
    def end_time(self):
        return self.beat.end_time()
    
    def __repr__(self):
        return f'B({self.beat}) P({self.pitch})'
    
    @classmethod
    def parse(cls, d, mode, key_offset):
        parsed = roman_to_symbol.hnote_parser(d, mode, key_offset)
        pitch = HPitch.parse_abs_pitch(parsed['pitch'], base_octave=config.note_octave) # pitch can be negative so offset octave
        beat = HBeat.parse_note(d)
        return cls(beat=beat, pitch=pitch) 

def remove_embelishment(d):
    d = d.copy()
    d['sus'] = None
    d['emb'] = None
    return d

def remove_9_11_chords(d):
    d = d.copy()
    if isinstance(d.get('fb'), str) and int(d['fb'][0]) > 7: d['fb'] = '9'
    return d
    
def shift_octave(parsed):
    lowest = min(parsed['composition'])
    if lowest >= 12:
        parsed = parsed.copy()
        parsed = roman_to_symbol.chord_key_shifting(parsed, -(lowest//12 * 12))
        # reset chord name
        new_s = roman_to_symbol.chord_to_string(parsed)
        parsed['symbol'] = new_s
    return parsed

@dataclass
class HChord(Base):
    def __init__(self, beat:HBeat, pitches:List[HPitch], inv:int,
                 composition:List[int]=None, symbol:str=None, quality:str=None, metadata=None, **kwargs):
        self.beat = beat
        self.inv = inv
        self.pitches = pitches
        self.composition, self.symbol, self.quality = composition, symbol, quality
        self.metadata = None
        
    def end_time(self):
        return self.beat.end_time()
        
    def to_m21(self)->music21.chord.Chord:
        notes = [p.abs(base_octave=config.chord_octave) for p in self.pitches]
        c = music21.chord.Chord(notes, quarterLength=self.beat.duration)
        return c, float(self.beat.abs)
    
    def __repr__(self):
        return f'B({self.beat}) C([{self.pitches}])'# + ' Comp:' + str(self.composition)

    @classmethod
    def parse(cls, d, mode, key_offset, reset_to_base=True, remove_emb=False, remove_large=True):
        if remove_emb: d = remove_embelishment(d)
        if remove_large: remove_9_11_chords(d)
        if d.get('emb') == 'add4': d['emb'] = None
            
            
        parsed_inv = roman_to_symbol.hchord_parser(d, mode, key_offset, invert=True)
        parsed_inv = shift_octave(parsed_inv)
        parsed_inv['composition'] = parsed_inv['composition'].astype(int).tolist()
        
        parsed = roman_to_symbol.hchord_parser(d, mode, key_offset, invert=False)
        # After offset, let's reset the chord to be the lowest possible offset on new scale
        if reset_to_base: parsed = shift_octave(parsed)
        parsed['composition'] = parsed['composition'].astype(int).tolist()
        
        beat = HBeat.parse_chord(d)
        pitches = [HPitch.parse_abs_pitch(p) for p in parsed['composition']] # first pitch = base note
        
        pitches_inv = [HPitch.parse_abs_pitch(p) for p in parsed_inv['composition']] # first pitch = base note
        
        pitches_shi = [HPitch.parse_abs_pitch(p) for p in parsed['composition']] # first pitch = base note
        for pinv in pitches_shi[:parsed['inv']]: pinv.octave += 1
           
        if parsed_inv['inv'] > 0:
#             print('Pnoi', pitches)
            print('Ptru', pitches_inv)
            print('Pshi', pitches_shi)
        
        
        return cls(beat=beat, pitches=pitches, metadata=parsed, **parsed)

def default_stream(cls=music21.stream.Score, ts='4/4', bpm=120, ks=0):
    # (AS) TODO: use config ts or metadata
    s = cls()
    s.append(music21.instrument.Piano())
    s.append(music21.meter.TimeSignature(ts))
    s.append(music21.tempo.MetronomeMark(number=bpm))
#     s.append(music21.key.KeySignature(ks))
    s.append(music21.key.Key('C'))
    return s

@dataclass
class HPart(Base):
    notes: List[HNote]
    chords: List[HChord]
        
    @classmethod
    def parse(cls, d, metadata):
        mode = metadata['mode'] or '1'
        key_offset = MODE_TO_KEYOFFSET.get(mode, 0)
        ns = [HNote.parse(n, mode, key_offset) for n in d.get('notes', []) if n['scale_degree'] != 'rest']
        cs = [HChord.parse(c, mode, key_offset) for c in d.get('chords', []) if c['sd'] != 'rest']
        ns = sorted(ns, key=lambda n: n.end_time())
        cs = sorted(cs, key=lambda c: c.end_time())
        return cls(notes=ns, chords=cs)
    
    def __repr__(self):
        chords = '[Chords]:\n' + '\n'.join([str(c) for c in self.chords])
        notes = '[Notes]:\n' + '\n'.join([str(n) for n in self.notes])
        return chords + '\n\n' + notes
    
    def duration(self):
        c_last = self.chords[-1].end_time() if self.chords else 0
        n_last = self.notes[-1].end_time() if self.notes else 0
        return max(c_last, n_last)
    
    # Note: first chord not played - https://github.com/rism-ch/verovio/issues/995
    def to_m21(self)->music21.stream.Stream:
        mc = music21.stream.Part()
        mn = music21.stream.Part()
        
        cm21 = [c.to_m21() for c in self.chords]
        for c,d in cm21: mc.insert(d,c)
            
        nm21 = [n.to_m21() for n in self.notes]
        for n,d in nm21: mn.insert(d,n)
        return mn, mc
        
    def min_pitch(self):
        return min([n.pitch for n in self.notes])

@dataclass
class HSong(Base):
    metadata: HMetadata
    parts: List[HPart]
    
    @classmethod
    def parse(cls, metadata, segments):
        m = HMetadata.parse(metadata)
        ps = [HPart.parse(s, metadata) for s in segments]
        return cls(metadata=m, parts=ps)
    
    def duration(self):
        return sum(p.duration() for p in self.parts)
    
    
    def __repr__(self):
        parts = ''
        for idx,p in enumerate(self.parts):
            parts += f'Part[{idx}]:\n' + str(p) + '\n\n'
        return parts + '\n\n' + str(self.metadata)
    
    def to_stream(self):
        s = default_stream()
        pc = music21.stream.Part()
        pn = music21.stream.Part()
        
        for p in self.parts:
            mn, mc = p.to_m21()
            pn.append(mn)
            pc.append(mc)
            
        s.insert(0.0, pn)
        s.insert(0.0, pc)

#         s.flat.makeNotation(inPlace=True)
        s = s.transpose(0) # hack to get accidentals right. Above method does not work
        # music21 stream
        return s
    

In [61]:
from src import roman_to_symbol
from src import to_pianoroll
from collections import defaultdict
from midi_data import keyc_offset

In [62]:
import numpy as np
from collections import Counter, defaultdict

In [63]:
from src.roman_to_symbol import *

In [64]:
def parse_file(file_path):
    content = load_data(file_path)
    root = xml_parser(content)
    metadata, version = get_metadata(root)
    segments, num_measures = get_lead_sheet(root, version)
    
    song = HSong.parse(metadata, segments)
    return song

In [65]:
all_symbols = []
all_notes = []
all_map = []
jumps = defaultdict(list)

In [66]:
note_octaves = []

In [67]:
file_path = Path('data/midi/v7/midi_sources/hooktheory/xml/w/wise-guys/du-doof/pre-chorus-and-chorus.xml')
s = parse_file(file_path)

Ptru [sd=6:oct=0, sd=1:oct=1, sd=3:oct=1, sd=5:oct=0]
Pshi [sd=6:oct=1, sd=1:oct=2, sd=3:oct=2, sd=5:oct=1]
Ptru [sd=2:oct=1, sd=4#:oct=1, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=2:oct=1, sd=4#:oct=1, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=1:oct=1, sd=2#:oct=1, sd=5:oct=1, sd=6#:oct=0]
Pshi [sd=1:oct=1, sd=2#:oct=1, sd=5:oct=1, sd=6#:oct=0]
Ptru [sd=2:oct=1, sd=4#:oct=1, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=2:oct=1, sd=4#:oct=1, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=6:oct=0, sd=1:oct=1, sd=3:oct=1, sd=5:oct=0]
Pshi [sd=6:oct=1, sd=1:oct=2, sd=3:oct=2, sd=5:oct=1]
Ptru [sd=2:oct=1, sd=4#:oct=1, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=2:oct=1, sd=4#:oct=1, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=1:oct=1, sd=3:oct=1, sd=5:oct=1, sd=6#:oct=0]
Pshi [sd=1:oct=1, sd=3:oct=1, sd=5:oct=1, sd=6#:oct=0]
Ptru [sd=2:oct=1, sd=4#:oct=1, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=2:oct=1, sd=4#:oct=1, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=6:oct=0, sd=1:oct=1, sd=3:oct=1, sd=5:oct=0]
Pshi [sd=6:oct=1, sd=1:oct=2, sd=3:oct=2, sd=5:oct=1]
Ptru [sd=2:oct

In [68]:
[c.composition for c in s.parts[0].chords]

[[9, 12, 16, 19],
 [2, 6, 9, 12],
 [0, 3, 7, 10],
 [2, 6, 9, 12],
 [9, 12, 16, 19],
 [2, 6, 9, 12],
 [0, 4, 7, 10],
 [2, 6, 9, 12],
 [4, 8, 11, 14],
 [2, 6, 9, 12],
 [11, 15, 18, 21],
 [4, 8, 11, 14]]

In [69]:
# file_path = Path('data/midi/v7/midi_sources/hooktheory/xml/w/wyd-krakow-2016/blogoslawieni-milosierni/verse.xml')
# content = load_data(file_path)
# root = xml_parser(content)
# metadata, version = get_metadata(root)
# segments, num_measures = get_lead_sheet(root, version)
# cdata['sus'] = None
# cdata['emb'] = 'add9'
# cdata = segments[-1]['chords'][-2]; cdata

In [70]:
from tqdm import tqdm

In [71]:
for f in tqdm(files, total=len(files)):
    try: s = parse_file(f)
    except Exception as e: 
        print('Exception:', e)
        continue
    no = []
    for p in s.parts:
        for n in p.notes:
            no.append(n.pitch.octave)
#         for c in p.chords:
#             if len(c.composition) > 4: continue
#             all_symbols.append(c.symbol)
#             for n in c.composition:
#                 if n > 24: print('Note out of bounds:', c.composition, f)
#             chords = list(([i%12 for i in sorted(c.composition)]))
#             all_notes.append(str(chords))
#             all_map.append(c.symbol + ' - ' + str(chords))
#             j = np.diff(sorted(c.composition))
#             jumps[str(j)].append((c.composition, c.symbol))
    note_octaves.append(no)
    if max(no, default=0) - min(no, default=0) > 4:
        print('Large octave spread:', f)

  0%|          | 11/19918 [00:00<03:21, 98.76it/s]

Ptru [sd=6:oct=0, sd=1:oct=1, sd=3:oct=0, sd=5:oct=0]
Pshi [sd=6:oct=1, sd=1:oct=2, sd=3:oct=1, sd=5:oct=1]
Ptru [sd=6:oct=0, sd=1:oct=1, sd=3:oct=1, sd=5:oct=0]
Pshi [sd=6:oct=1, sd=1:oct=2, sd=3:oct=2, sd=5:oct=1]
Ptru [sd=7:oct=0, sd=2:oct=1, sd=4#:oct=1, sd=6:oct=0]
Pshi [sd=7:oct=1, sd=2:oct=2, sd=4#:oct=2, sd=6:oct=1]
Ptru [sd=2:oct=1, sd=4:oct=0, sd=5#:oct=0, sd=1:oct=1]
Pshi [sd=2:oct=1, sd=4:oct=0, sd=5#:oct=0, sd=1:oct=1]
Ptru [sd=1:oct=1, sd=3:oct=0, sd=5:oct=0]
Pshi [sd=1:oct=1, sd=3:oct=0, sd=5:oct=0]
Ptru [sd=6#:oct=0, sd=2:oct=1, sd=4:oct=0]
Pshi [sd=6#:oct=1, sd=2:oct=2, sd=4:oct=1]
Ptru [sd=1:oct=1, sd=3:oct=1, sd=5:oct=0]
Pshi [sd=1:oct=1, sd=3:oct=1, sd=5:oct=0]
Ptru [sd=1:oct=1, sd=3:oct=0, sd=5:oct=0]
Pshi [sd=1:oct=1, sd=3:oct=0, sd=5:oct=0]
Ptru [sd=6#:oct=0, sd=2:oct=1, sd=4:oct=0]
Pshi [sd=6#:oct=1, sd=2:oct=2, sd=4:oct=1]
Ptru [sd=1:oct=1, sd=3:oct=1, sd=5:oct=0]
Pshi [sd=1:oct=1, sd=3:oct=1, sd=5:oct=0]
Ptru [sd=1:oct=1, sd=3:oct=0, sd=5:oct=0]
Pshi [sd=1:oct

  0%|          | 23/19918 [00:00<04:54, 67.51it/s]

[sd=5:oct=1, sd=7:oct=0, sd=2:oct=1]
Pshi [sd=5:oct=1, sd=7:oct=0, sd=2:oct=1]
Ptru [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=1:oct=1, sd=3:oct=1, sd=5:oct=0]
Pshi [sd=1:oct=1, sd=3:oct=1, sd=5:oct=0]
Ptru [sd=2:oct=1, sd=4#:oct=0, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=2:oct=1, sd=4#:oct=0, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=2:oct=1, sd=4:oct=0, sd=6:oct=0]
Pshi [sd=2:oct=1, sd=4:oct=0, sd=6:oct=0]
Ptru [sd=1:oct=1, sd=3:oct=0, sd=5:oct=0]
Pshi [sd=1:oct=1, sd=3:oct=0, sd=5:oct=0]
Ptru [sd=5:oct=0, sd=7:oct=0, sd=2:oct=0, sd=4:oct=0]
Pshi [sd=5:oct=1, sd=7:oct=1, sd=2:oct=1, sd=4:oct=1]
Ptru [sd=1:oct=1, sd=3:oct=0, sd=5:oct=0]
Pshi [sd=1:oct=1, sd=3:oct=0, sd=5:oct=0]
Ptru [sd=1:oct=1, sd=3:oct=1, sd=5:oct=0]
Pshi [sd=1:oct=1, sd=3:oct=1, sd=5:oct=0]
Ptru [sd=2:oct=1, sd=4#:oct=0, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=2:oct=1, sd=4#:oct=0, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=5:oct=1, sd=7:oct=0, sd=2:oct=1, sd=4:oct=1]
Pshi [sd=5:oct=1, sd=7:oct=0, sd=2:

  0%|          | 40/19918 [00:00<04:26, 74.46it/s]

...

  2%|▏         | 468/19918 [00:06<05:25, 59.68it/s]

[sd=1:oct=1, sd=3:oct=0, sd=5:oct=0]
Ptru [sd=5:oct=1, sd=7:oct=0, sd=2:oct=1]
Pshi [sd=5:oct=1, sd=7:oct=0, sd=2:oct=1]
Ptru [sd=5:oct=1, sd=7:oct=0, sd=2:oct=1]
Pshi [sd=5:oct=1, sd=7:oct=0, sd=2:oct=1]
Ptru [sd=3:oct=1, sd=5:oct=0, sd=7:oct=0]
Pshi [sd=3:oct=1, sd=5:oct=0, sd=7:oct=0]
Ptru [sd=5:oct=1, sd=7:oct=0, sd=2:oct=1, sd=6:oct=1]
Pshi [sd=5:oct=1, sd=7:oct=0, sd=2:oct=1, sd=6:oct=1]
Ptru [sd=5:oct=0, sd=7:oct=0, sd=2:oct=0]
Pshi [sd=5:oct=1, sd=7:oct=1, sd=2:oct=1]
Ptru [sd=5:oct=1, sd=7:oct=0, sd=2:oct=1, sd=6:oct=1]
Pshi [sd=5:oct=1, sd=7:oct=0, sd=2:oct=1, sd=6:oct=1]
Ptru [sd=5:oct=0, sd=7:oct=0, sd=2:oct=0]
Pshi [sd=5:oct=1, sd=7:oct=1, sd=2:oct=1]
Ptru [sd=5:oct=1, sd=6#:oct=0, sd=2:oct=1, sd=6:oct=1]
Pshi [sd=5:oct=1, sd=6#:oct=0, sd=2:oct=1, sd=6:oct=1]
Ptru [sd=2:oct=1, sd=4:oct=1, sd=6:oct=0]
Pshi [sd=2:oct=1, sd=4:oct=1, sd=6:oct=0]
Ptru [sd=2:oct=1, sd=4:oct=1, sd=6:oct=0]
Pshi [sd=2:oct=1, sd=4:oct=1, sd=6:oct=0]
Ptru [sd=4:oct=0, sd=5#:oct=0, sd=1:oct=0]
Pshi [

  2%|▏         | 480/19918 [00:06<04:42, 68.88it/s]

Ptru [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=3:oct=1, sd=5:oct=0, sd=7:oct=0, sd=2:oct=1]
Pshi [sd=3:oct=1, sd=5:oct=0, sd=7:oct=0, sd=2:oct=1]
Ptru [sd=2:oct=1, sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Pshi [sd=2:oct=1, sd=4:oct=1, sd=6:oct=0, sd=1:oct=1]
Ptru [sd=3:oct=1, sd=5:oct=0, sd=7:oct=0, sd=2:oct=1]
Pshi [sd=3:oct=1, sd=5:oct=0, sd=7:oct=0, sd=2:oct=1]
Ptru

KeyboardInterrupt: 

In [40]:
min([], default=0)

0

In [43]:
max([max(no, default=0) - min(no, default=0) for no in note_octaves])

5

In [41]:
[(min(no, default=0), max(no, default=0)) for no in note_octaves]

[(3, 5),
 (5, 5),
 (5, 6),
 (5, 6),
 (5, 6),
 (5, 6),
 (4, 5),
 (4, 5),
 (5, 5),
 (5, 5),
 (3, 3),
 (4, 6),
 (0, 0),
 (0, 0),
 (4, 5),
 (5, 5),
 (4, 6),
 (4, 6),
 (4, 6),
 (5, 6),
 (5, 6),
 (4, 5),
 (4, 5),
 (5, 6),
 (6, 7),
 (6, 7),
 (6, 7),
 (4, 5),
 (5, 6),
 (4, 6),
 (4, 5),
 (0, 0),
 (5, 6),
 (5, 6),
 (4, 5),
 (5, 6),
 (5, 6),
 (6, 7),
 (5, 7),
 (4, 5),
 (5, 6),
 (5, 5),
 (5, 6),
 (5, 5),
 (4, 5),
 (5, 5),
 (3, 4),
 (5, 5),
 (5, 5),
 (4, 6),
 (4, 5),
 (5, 6),
 (5, 6),
 (5, 6),
 (5, 6),
 (5, 5),
 (3, 4),
 (3, 4),
 (0, 0),
 (3, 4),
 (5, 5),
 (4, 5),
 (4, 5),
 (0, 0),
 (4, 5),
 (4, 5),
 (4, 5),
 (4, 5),
 (4, 5),
 (3, 7),
 (3, 5),
 (4, 5),
 (3, 6),
 (4, 5),
 (4, 7),
 (4, 5),
 (4, 4),
 (3, 4),
 (5, 5),
 (4, 5),
 (5, 5),
 (4, 5),
 (0, 0),
 (5, 6),
 (4, 5),
 (3, 5),
 (5, 6),
 (3, 5),
 (3, 4),
 (4, 5),
 (4, 6),
 (4, 5),
 (4, 7),
 (5, 5),
 (4, 6),
 (4, 5),
 (4, 5),
 (4, 6),
 (3, 5),
 (4, 5),
 (4, 5),
 (3, 4),
 (4, 5),
 (5, 5),
 (4, 5),
 (4, 5),
 (4, 5),
 (4, 5),
 (5, 5),
 (4, 5),
 (0, 0),
 

In [ ]:
jumps_count = {k:len(v) for k,v in jumps.items()}; len(jumps_count)

In [ ]:
jumps['[4 1 4]']

In [32]:
len(all_symbols)

316795

In [34]:
len(set(all_symbols))

287

In [36]:
symbols = set(all_symbols); symbols

{'A',
 'A | Db',
 'A | E',
 'A7',
 'A7 | Db',
 'A7 | E',
 'A7 | G',
 'Ab',
 'Ab | C',
 'Ab | Eb',
 'Ab7',
 'Ab7 | C',
 'Ab7 | Eb',
 'Ab7 | Gb',
 'Abmaj7',
 'Abmaj7 | C',
 'Abmaj7 | Eb',
 'Abmaj7 | G',
 'Amaj7',
 'Amaj7 | Ab',
 'Amaj7 | E',
 'B',
 'B | Eb',
 'B | Gb',
 'B7',
 'B7 | A',
 'B7 | Eb',
 'B7 | Gb',
 'Bb',
 'Bb | D',
 'Bb | F',
 'Bb7',
 'Bb7 | Ab',
 'Bb7 | D',
 'Bb7 | F',
 'Bbmaj7',
 'Bbmaj7 | A',
 'Bbmaj7 | D',
 'Bbmaj7 | F',
 'Bmaj7',
 'Bmaj7 | Bb',
 'Bmaj7 | Eb',
 'Bmaj7 | Gb',
 'C',
 'C | E',
 'C | G',
 'C7',
 'C7 | Bb',
 'C7 | E',
 'C7 | G',
 'Cmaj7',
 'Cmaj7 | B',
 'Cmaj7 | E',
 'Cmaj7 | G',
 'D',
 'D | A',
 'D | Gb',
 'D7',
 'D7 | A',
 'D7 | C',
 'D7 | Gb',
 'Db',
 'Db | Ab',
 'Db | F',
 'Db7',
 'Db7 | Ab',
 'Db7 | B',
 'Db7 | F',
 'Dbmaj7',
 'Dbmaj7 | Ab',
 'Dbmaj7 | C',
 'Dbmaj7 | F',
 'Dmaj7',
 'Dmaj7 | A',
 'Dmaj7 | Db',
 'E',
 'E | Ab',
 'E | B',
 'E7',
 'E7 | Ab',
 'E7 | B',
 'E7 | D',
 'Eb',
 'Eb | Bb',
 'Eb | G',
 'Eb7',
 'Eb7 | Bb',
 'Eb7 | Db',
 'Eb7 | G',
 'E

In [39]:
no_inv = [s for s in symbols if ' | ' not in s]; len(no_inv)

84

In [40]:
no_inv

['Eb',
 'eo',
 'F7',
 'Bmaj7',
 'co',
 'D',
 'bø7',
 'Fmaj7',
 'do',
 'dbm',
 'Dbmaj7',
 'D7',
 'F',
 'dbm7',
 'Abmaj7',
 'Bbmaj7',
 'Cmaj7',
 'B7',
 'ebo',
 'dm',
 'ao',
 'Ab7',
 'gbm7',
 'bbø7',
 'dm7',
 'Gb7',
 'Db',
 'bbo',
 'Amaj7',
 'ebø7',
 'fø7',
 'go',
 'bm7',
 'Bb',
 'dbo',
 'eø7',
 'em7',
 'am',
 'E',
 'aø7',
 'em',
 'abø7',
 'bbm',
 'gbø7',
 'cm7',
 'Ab',
 'ebm7',
 'B',
 'Db7',
 'am7',
 'E7',
 'A',
 'abo',
 'Ebmaj7',
 'A7',
 'gø7',
 'cø7',
 'fo',
 'bbm7',
 'gbm',
 'bo',
 'dø7',
 'C',
 'Dmaj7',
 'bm',
 'Gb',
 'abm7',
 'abm',
 'ebm',
 'G7',
 'gm',
 'Gmaj7',
 'cm',
 'fm',
 'Bb7',
 'gbo',
 'C7',
 'gm7',
 'Emaj7',
 'fm7',
 'Gbmaj7',
 'Eb7',
 'G',
 'dbø7']

In [42]:
no_inv_7 = [s for s in no_inv if '7' not in s]; len(no_inv_7)

36

In [43]:
no_inv_7

['Eb',
 'eo',
 'co',
 'D',
 'do',
 'dbm',
 'F',
 'ebo',
 'dm',
 'ao',
 'Db',
 'bbo',
 'go',
 'Bb',
 'dbo',
 'am',
 'E',
 'em',
 'bbm',
 'Ab',
 'B',
 'A',
 'abo',
 'fo',
 'gbm',
 'bo',
 'C',
 'bm',
 'Gb',
 'abm',
 'ebm',
 'gm',
 'cm',
 'fm',
 'gbo',
 'G']

In [37]:
Counter(all_symbols)

Counter({'em': 4856,
         'am7 | E': 59,
         'gm': 6209,
         'am7 | G': 407,
         'bm7 | A': 13,
         'cm': 33977,
         'dø7 | F': 294,
         'ebm': 478,
         'cm7': 4481,
         'Cmaj7': 2435,
         'Ab': 17270,
         'Eb': 11450,
         'Bb': 20217,
         'F7': 724,
         'G': 27719,
         'C': 39422,
         'am': 14355,
         'dm': 6761,
         'C | E': 3314,
         'Bb | F': 738,
         'F': 28348,
         'dm7': 3683,
         'C | G': 2369,
         'do': 483,
         'G | D': 1216,
         'E': 1238,
         'E7': 613,
         'D': 2503,
         'D7': 865,
         'G7': 4831,
         'C7': 1352,
         'gbo': 502,
         'A7': 558,
         'am7': 3109,
         'bbø7': 2,
         'G7 | B': 374,
         'Gb7': 49,
         'A7 | E': 25,
         'dm7 | F': 459,
         'Cmaj7 | E': 118,
         'Fmaj7': 3040,
         'Fmaj7 | A': 57,
         'bø7': 155,
         'bø7 | D': 51,
         'E7 | Ab': 11

In [31]:
all_symbols

['em',
 'am7 | E',
 'em',
 'gm',
 'am7 | G',
 'bm7 | A',
 'cm',
 'dø7 | F',
 'cm',
 'ebm',
 'cm7',
 'Cmaj7',
 'cm7',
 'Cmaj7',
 'Ab',
 'Eb',
 'Ab',
 'Bb',
 'cm7',
 'Ab',
 'Eb',
 'F7',
 'G',
 'cm',
 'Ab',
 'Bb',
 'cm',
 'Ab',
 'Bb',
 'cm',
 'Ab',
 'Bb',
 'Bb',
 'cm',
 'Ab',
 'Bb',
 'cm',
 'Ab',
 'Bb',
 'Eb',
 'cm',
 'Ab',
 'Bb',
 'Eb',
 'cm',
 'Ab',
 'Bb',
 'Eb',
 'Ab',
 'gm',
 'Ab',
 'Bb',
 'cm',
 'cm',
 'Ab',
 'Bb',
 'Eb',
 'cm',
 'Ab',
 'Bb',
 'Eb',
 'cm',
 'Ab',
 'Bb',
 'Eb',
 'Ab',
 'gm',
 'Ab',
 'Bb',
 'cm',
 'C',
 'am',
 'dm',
 'G',
 'C',
 'am',
 'dm',
 'G',
 'C',
 'am',
 'dm',
 'G',
 'C',
 'am',
 'dm',
 'G',
 'C',
 'C | E',
 'Bb | F',
 'F',
 'dm7',
 'C | G',
 'G',
 'G',
 'C',
 'C | E',
 'Bb | F',
 'F',
 'dm7',
 'C | G',
 'G',
 'G',
 'cm',
 'cm',
 'cm',
 'cm',
 'cm',
 'do',
 'do',
 'do',
 'do',
 'do',
 'cm',
 'cm',
 'cm',
 'cm',
 'cm',
 'G',
 'G',
 'G',
 'G',
 'G',
 'C',
 'F',
 'C',
 'F',
 'C',
 'F',
 'G',
 'G',
 'F',
 'C',
 'Bb',
 'C',
 'F',
 'C',
 'F',
 'G',
 'am',
 'am',
 'G',

In [143]:
sum(jumps_count.values())

316795

In [144]:
{k:v/316795 for k,v in jumps_count.items()}

{'[3 4]': 0.24016161871241654,
 '[3 2 3]': 0.0016951025110876119,
 '[2 3 4]': 0.005814485708423429,
 '[3 4 2]': 0.0020707397528370083,
 '[3 4 3]': 0.061326725484935056,
 '[4 3 4]': 0.038220300194131855,
 '[4 3]': 0.48079672974636595,
 '[4 3 3]': 0.03310973973705393,
 '[3 5]': 0.03778153064284474,
 '[5 4]': 0.02551176628419009,
 '[3 3]': 0.010527312615413753,
 '[3 3 4]': 0.0037216496472482203,
 '[3 3 2]': 0.0029735317792263134,
 '[3 2 4]': 0.002130715446897836,
 '[4 3 2]': 0.008728041793588914,
 '[3 4 1]': 0.0013510314241070723,
 '[6 3]': 0.002001294212345523,
 '[1 4 3]': 0.002373774838618034,
 '[2 4 3]': 0.004883284142742152,
 '[4 5]': 0.016660616487002636,
 '[5 3]': 0.013021038842153443,
 '[3 6]': 0.0019413185182846951,
 '[4 2 3]': 0.0010132735680803042,
 '[2 3 3]': 0.0011426948026326174,
 '[4 1 4]': 0.0010416831073722754}

In [137]:
jumps_count

{'[3 4]': 76082,
 '[3 2 3]': 537,
 '[2 3 4]': 1842,
 '[3 4 2]': 656,
 '[3 4 3]': 19428,
 '[4 3 4]': 12108,
 '[4 3]': 152314,
 '[4 3 3]': 10489,
 '[3 5]': 11969,
 '[5 4]': 8082,
 '[3 3]': 3335,
 '[3 3 4]': 1179,
 '[3 3 2]': 942,
 '[3 2 4]': 675,
 '[4 3 2]': 2765,
 '[3 4 1]': 428,
 '[6 3]': 634,
 '[1 4 3]': 752,
 '[2 4 3]': 1547,
 '[4 5]': 5278,
 '[5 3]': 4125,
 '[3 6]': 615,
 '[4 2 3]': 321,
 '[2 3 3]': 362,
 '[4 1 4]': 330}

In [138]:
# quality - 

In [139]:
symbol_count = Counter(all_symbols); len(symbol_count)

287

In [140]:
symbol_count.most_common()

[('C', 39422),
 ('cm', 33977),
 ('F', 28348),
 ('G', 27719),
 ('Bb', 20217),
 ('Ab', 17270),
 ('am', 14355),
 ('Eb', 11450),
 ('fm', 8082),
 ('dm', 6761),
 ('gm', 6209),
 ('em', 4856),
 ('G7', 4831),
 ('cm7', 4481),
 ('Abmaj7', 4199),
 ('dm7', 3683),
 ('C | E', 3314),
 ('G | B', 3179),
 ('am7', 3109),
 ('Fmaj7', 3040),
 ('fm7', 2727),
 ('gm7', 2605),
 ('D', 2503),
 ('Cmaj7', 2435),
 ('C | G', 2369),
 ('em7', 2066),
 ('Db', 1992),
 ('cm | Eb', 1738),
 ('F | C', 1718),
 ('cm | G', 1714),
 ('F | A', 1494),
 ('C7', 1352),
 ('Bb | D', 1269),
 ('E', 1238),
 ('G | D', 1216),
 ('Ebmaj7', 1008),
 ('Eb | G', 973),
 ('Eb | Bb', 966),
 ('gm | Bb', 958),
 ('A', 958),
 ('bo', 930),
 ('fm | Ab', 897),
 ('D7', 865),
 ('Ab | C', 819),
 ('Bb7', 750),
 ('Bb | F', 738),
 ('cm7 | Bb', 730),
 ('F7', 724),
 ('bbm', 657),
 ('Gb', 633),
 ('fm | C', 619),
 ('E7', 613),
 ('gm7 | Bb', 593),
 ('Ab | Eb', 581),
 ('B', 564),
 ('A7', 558),
 ('am | C', 537),
 ('gbo', 502),
 ('Dbmaj7', 501),
 ('dm | F', 496),
 ('do', 4

In [141]:
note_count = Counter(all_notes); len(note_count)

287

In [142]:
note_count.most_common()

[('[0, 4, 7]', 39422),
 ('[0, 3, 7]', 33977),
 ('[5, 9, 0]', 28348),
 ('[7, 11, 2]', 27719),
 ('[10, 2, 5]', 20217),
 ('[8, 0, 3]', 17270),
 ('[9, 0, 4]', 14355),
 ('[3, 7, 10]', 11450),
 ('[5, 8, 0]', 8082),
 ('[2, 5, 9]', 6761),
 ('[7, 10, 2]', 6209),
 ('[4, 7, 11]', 4856),
 ('[7, 11, 2, 5]', 4831),
 ('[0, 3, 7, 10]', 4481),
 ('[8, 0, 3, 7]', 4199),
 ('[2, 5, 9, 0]', 3683),
 ('[4, 7, 0]', 3314),
 ('[11, 2, 7]', 3179),
 ('[9, 0, 4, 7]', 3109),
 ('[5, 9, 0, 4]', 3040),
 ('[5, 8, 0, 3]', 2727),
 ('[7, 10, 2, 5]', 2605),
 ('[2, 6, 9]', 2503),
 ('[0, 4, 7, 11]', 2435),
 ('[7, 0, 4]', 2369),
 ('[4, 7, 11, 2]', 2066),
 ('[1, 5, 8]', 1992),
 ('[3, 7, 0]', 1738),
 ('[0, 5, 9]', 1718),
 ('[7, 0, 3]', 1714),
 ('[9, 0, 5]', 1494),
 ('[0, 4, 7, 10]', 1352),
 ('[2, 5, 10]', 1269),
 ('[4, 8, 11]', 1238),
 ('[2, 7, 11]', 1216),
 ('[3, 7, 10, 2]', 1008),
 ('[7, 10, 3]', 973),
 ('[10, 3, 7]', 966),
 ('[10, 2, 7]', 958),
 ('[9, 1, 4]', 958),
 ('[11, 2, 5]', 930),
 ('[8, 0, 5]', 897),
 ('[2, 6, 9, 0]', 

chord without inversion

In [ ]:


# def hchord_parser(chord, mode, key_offset):
#     if chord['sd'] == 'rest': return None

#     # extract basic info
#     sd = int(chord['sd']) - 1     # root
#     fb = chord['fb']              # tension & inversion
#     sec = chord['sec']            # secondary chord
#     borrowed = chord.get('borrowed', None)  # borrowed mode

#     # determine the mode
#     borrowed = is_int(borrowed)
#     chord_key = MODE_TO_KEY[int(mode)] if borrowed is None else borrowed
#     chord_key = 6 if chord_key > 6 else chord_key
#     chord_key = -6 if chord_key < -6 else chord_key

#     # secondary chord
#     sec_offset = 0
#     if sec:
#         # switch to 'sec' degree note within the current mode
#         scale = KEY_TO_SCALE[MODE_TO_KEY[int(mode)]]
#         new_key_note = scale[int(sec) - 1]

#         # set that note to new key
#         new_key = VAL_TO_NAME[new_key_note][0]

#         # get the key shift offset
#         sec_offset = NOTE_TO_OFFSET[new_key]
#         chord_key = 0

#     # determine the scale according to the key(mode) of the chord
#     scale = get_scale(chord_key)

#     # set compositional notes
#     comp, chord_type = set_compositions(scale, fb, sd)

#     # determine the quality by triads or seventh
#     # (9, 11, 13-th chords are seen as seventh)
#     comp_t = comp[0:3] if chord_type is 5 else comp[0:4]
#     quality = get_quality(comp_t)

#     # add shift from secondary chords
#     comp = (comp + sec_offset)

#     # set compvec (for sus/add/omit)
#     comp_vec = comp_to_compvec(comp)

#     # sus (omit 3)
#     sus = chord.get('sus', None)
#     comp_vec = set_sus(comp_vec, scale, sd, sus)

#     # emb (add/omit)
#     if 'emb' in chord:
#         emb = chord['emb']
#         comp_vec, alter_info, emb_info = set_emb(comp_vec, scale, sd, emb)
#     else:
#         emb_info = []
#         alter_info = []

#     # alter (won't change the quality)
#     alter_info = alter_info if len(alter_info) else chord.get('alternate']
#     comp_vec, alter_map = set_alter(comp_vec, alter_info)

#     # set inversion (won't change the root, but bass)
#     inv = get_num_inversion(fb)
#     comp_vec = set_inversion(comp_vec, inv)

#     # set result
#     comp = compvec_to_comp(comp_vec)
#     root = (comp[0] + 120) % 12   # for chord name
#     bass = np.nanmin(comp)         # for bass (real root)

#     data = OrderedDict([
#         # basic compositions
#         ('root', root),
#         ('bass', bass),
#         ('comp_vec', comp_vec),
#         ('composition', comp),

#         # basic info
#         ('quality', quality),
#         ('chord_type', chord_type),
#         ('chord_mode', chord_key),
        
#         # additional info
#         ('inv', inv),
#         ('sus', sus),
#         ('alter', alter_info),
#         ('emb', emb_info),
#         ('alter_map', alter_map),
#         ])

#     # key shifting of the symbol
#     data = chord_key_shifting(data, key_offset)

#     # set chord name
#     data['symbol'] = chord_to_string(data)
#     return data

In [180]:
VAL_TO_NAME = {
    1: 
}

In [ ]:
def chord_to_string(data):
    base = to_name(data['root']) + data['quality']
    if data['quality'] in ['m', 'ø', 'o']:
        base = base.lower()
    base += ('' if data['chord_type'] is 5 else str(data['chord_type']))
    if data['inv']:
        base += (' | ' + to_name(data['bass']))
    if data['sus'] is not None:
        base += (' ' + data['sus'])
    if data['emb'] is not None:
        for e in data['emb']:
            base += ' (%s)' % e
    if data['alter'] is not None:
        for a in data['alter']:
            base += ' (%s)' % a
    return base